In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [6]:
df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,0.0
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,NaN
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,NaN
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,NaN
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,NaN
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,NaN


In [8]:
df.shape

(569, 33)

In [10]:
df.drop(columns=["id","Unnamed: 32"], inplace=True)

In [15]:
df.sample(7)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
127,M,19.00,18.91,123.40,1138.0,0.08217,0.08028,0.092710,0.056270,0.1946,0.05044,0.6896,1.342,5.216,81.23,0.004428,0.027310,0.040400,0.013610,0.02030,0.002686,22.32,25.73,148.20,1538.0,0.10210,0.22640,0.320700,0.121800,0.2841,0.06541
169,B,14.97,16.95,96.22,685.9,0.09855,0.07885,0.026020,0.037810,0.1780,0.05650,0.2713,1.217,1.893,24.28,0.005080,0.013700,0.007276,0.009073,0.01350,0.001706,16.11,23.00,104.60,793.7,0.12160,0.16370,0.066480,0.084850,0.2404,0.06428
452,B,12.00,28.23,76.77,442.5,0.08437,0.06450,0.040550,0.019450,0.1615,0.06104,0.1912,1.705,1.516,13.86,0.007334,0.025890,0.029410,0.009166,0.01745,0.004302,13.09,37.88,85.07,523.7,0.12080,0.18560,0.181100,0.071160,0.2447,0.08194
10,M,16.02,23.24,102.70,797.8,0.08206,0.06669,0.032990,0.033230,0.1528,0.05697,0.3795,1.187,2.466,40.51,0.004029,0.009269,0.011010,0.007591,0.01460,0.003042,19.19,33.88,123.80,1150.0,0.11810,0.15510,0.145900,0.099750,0.2948,0.08452
162,M,19.59,18.15,130.70,1214.0,0.11200,0.16660,0.250800,0.128600,0.2027,0.06082,0.7364,1.048,4.792,97.07,0.004057,0.022770,0.040290,0.013030,0.01686,0.003318,26.73,26.39,174.90,2232.0,0.14380,0.38460,0.681000,0.224700,0.3643,0.09223
124,B,13.37,16.39,86.10,553.5,0.07115,0.07325,0.080920,0.028000,0.1422,0.05823,0.1639,1.140,1.223,14.66,0.005919,0.032700,0.049570,0.010380,0.01208,0.004076,14.26,22.75,91.99,632.1,0.10250,0.25310,0.330800,0.089780,0.2048,0.07628
178,B,13.01,22.22,82.01,526.4,0.06251,0.01938,0.001595,0.001852,0.1395,0.05234,0.1731,1.142,1.101,14.34,0.003418,0.002252,0.001595,0.001852,0.01613,0.000968,14.00,29.02,88.18,608.8,0.08125,0.03432,0.007977,0.009259,0.2295,0.05843


### Train-Test Split

In [16]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

### Scaling

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Label Encoding on Target

In [20]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

### Numpy arrays to Pytorch tensors

In [21]:
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)
X_test_tensor = torch.from_numpy(X_test)
y_test_tensor = torch.from_numpy(y_test)

In [22]:
X_train_tensor.shape

torch.Size([455, 30])

### Defining Model

In [41]:
class MySimpleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1],1,dtype=torch.float64, requires_grad=True)
        self.bais = torch.zeros(1,dtype=torch.float64,requires_grad=True)

    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bais
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred, y):
        loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss
    

### Important Parameters

In [45]:
learning_rate = 0.1
epochs = 1000

### Training pipeline

In [46]:
# Create model
model = MySimpleNN(X_test_tensor)
model.weights

## define loop
for epoch in range(epochs):
    ## forward pass
    y_pred = model.forward(X_test_tensor)

    ## loss calculate
    loss = model.loss_function(y_pred, y_test_tensor)

    ## backward pass
    loss.backward()

    ## paramters update
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bais -= learning_rate * model.bais.grad

    ## zero gradients
    model.weights.grad.zero_()
    model.bais.grad.zero_()

    ## print loss in each epochs
    print(f'Epochs: {epochs + 1}, Loss: {loss.item()}')

Epochs: 1001, Loss: 4.070101670038381
Epochs: 1001, Loss: 3.859799999550753
Epochs: 1001, Loss: 3.648353237260537
Epochs: 1001, Loss: 3.4393388998591723
Epochs: 1001, Loss: 3.233024793279373
Epochs: 1001, Loss: 3.029677173529173
Epochs: 1001, Loss: 2.82969819107592
Epochs: 1001, Loss: 2.6335707418668095
Epochs: 1001, Loss: 2.4418499628401618
Epochs: 1001, Loss: 2.2551780591510546
Epochs: 1001, Loss: 2.0743036192974578
Epochs: 1001, Loss: 1.9001174602303446
Epochs: 1001, Loss: 1.73370371414193
Epochs: 1001, Loss: 1.5764001653945916
Epochs: 1001, Loss: 1.429847992081205
Epochs: 1001, Loss: 1.295988685279744
Epochs: 1001, Loss: 1.176926052361148
Epochs: 1001, Loss: 1.0745416258545293
Epochs: 1001, Loss: 0.9898654826701548
Epochs: 1001, Loss: 0.9225244951559133
Epochs: 1001, Loss: 0.8707145991916722
Epochs: 1001, Loss: 0.8317238851594102
Epochs: 1001, Loss: 0.8026171281890321
Epochs: 1001, Loss: 0.7807531676058082
Epochs: 1001, Loss: 0.7640384654657808
Epochs: 1001, Loss: 0.750958256240861

In [47]:
model.weights

tensor([[-0.2413],
        [-0.0847],
        [ 0.4987],
        [-0.0614],
        [-0.0301],
        [-0.1477],
        [-0.1466],
        [ 0.0360],
        [ 0.0739],
        [ 0.1480],
        [-0.2449],
        [-0.0048],
        [ 0.0353],
        [ 0.3198],
        [ 0.0391],
        [-0.1346],
        [ 0.0171],
        [-0.0198],
        [ 0.0240],
        [ 0.0531],
        [-0.3388],
        [ 0.0957],
        [-0.0940],
        [ 0.1956],
        [-0.0348],
        [ 0.1849],
        [ 0.0816],
        [ 0.1263],
        [-0.0908],
        [-0.1251]], dtype=torch.float64, requires_grad=True)

### Evaluation of model

In [50]:
with torch.no_grad():
    y_pred = model.forward(X_test_tensor)
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Test Accuracy: {accuracy.item()}")

Test Accuracy: 0.6228070259094238
